# Lab 04: Functional spatial image processing

The purpose of this lab is to practice registration steps that transform a functional image to higher-resolution anatomical space of the same subject, and then to standard space. We will then learn how to use these transforms to move an atlas-based mask from standard space to the subject's functional space.

## Preparation
* Import python tools for plotting 3D images
* Copy images from Lab 03 for our sub-97 example that we've been working with

In [ ]:
%matplotlib inline
from nilearn import plotting
from nilearn import image

In [ ]:
%%bash
# Copy files from previous labs (or download) so it is easier work with them here

# Files from 03 Lab that include our transforms for T1->MNI
# DECIDE WHETHER YOU WANT TO COPY (cp) OR DOWNLOAD THE FILES NECESSARY FOR THIS LAB
# default
ans="dl"

if [ "${ans}" == "cp" ]; then
    cp -t ./ \
       ../03-Lab/my_T1_brain.nii.gz \
       ../03-Lab/my_T1.nii.gz \
       ../03-Lab/MNI152_T1_2mm_brain.nii.gz \
       ../03-Lab/MNI152_T1_2mm.nii.gz \
       ../03-Lab/T1_2_MNI.mat \
       ../03-Lab/T1_2_MNI_warp.nii.gz
    cp ../01-Lab/osfshare/sub-97_task-flanker_bold.nii.gz my_bold.nii.gz # Copy/rename functional image from osfshare
elif [ "${ans}" == "dl" ]; then
    wget --quiet -O 04-Lab_data.tar.gz https://osf.io/4yz8m/download &&\
    tar -xf 04-Lab_data.tar.gz --strip-components=2 &&\
    rm 04-Lab_data.tar.gz
else
    printf 'YOU FORGOT TO CHANGE ANS TO "cp" OR "dl"'
fi


## Registration of functional image to T1

Summary of steps 
* Extract reference volume from the 4D functional image data set 
    * fslroi tool is handy for this
   
* Estimation: 
    * Boundary Based Registration (BBR) which in theory is most accurate but takes a bit of time to run (try at home)
    * 6 degrees of freedom (DOF) co-registration (faster, practice in class)

In [ ]:
# There are 315 volumes in this functional data set, it doesn't make a big deal whether you round up or down
315/2

In [ ]:
%%bash
fslroi

In [ ]:
%%bash

# Example usage to name our middle volume as example_func
# Reads: take input of my_bold, give back example func after extracting volumes starting at vol 157 with length 1
# Remember since indexing of volumes starts at zero, this is the 158th volume
fslroi my_bold.nii.gz example_func.nii.gz 157 1

In [ ]:
%%bash

# skull strip the reference epi so that skull information doesn't influence the registration to the T1w brain.
bet example_func.nii.gz example_func_brain.nii.gz

In [ ]:
%%bash

# Take a look with fslview
fslview example_func.nii.gz example_func_brain.nii.gz &


In [ ]:
%%bash

# Code for the 6 DOF func2T1 co-registration
flirt -in example_func_brain \
-ref my_T1_brain \
-out func2T1_6dof \
-omat func2T1_6dof.mat \
-cost corratio \
-dof 6 \
-searchrx -90 90 -searchry -90 90 -searchrz -90 90 \
-interp trilinear


In [ ]:
%%bash

# What did this produce? What is our main output of interest? 


## Concatenating transforms

For functional image analysis, we often need to align each individual's functional image with a standard space image. The standard space image (e.g., MNI 152 2mm) is a common reference space that we will align all our functional images to. This allows us to have a comparable image space for statistical analyses.  

So far we have:
* A transform from func to T1 
* A transform + warp from T1 to MNI 

We don't want to apply these sequentially *on the images themselves* because once the func is transformed we lose some precision in the image compared to the original. So we'll combine these transforms mathematically with FSL's convertwarp tool. 

In [ ]:
%%bash

# Next we need to use convertwarp to concatenate our affine with our nonlinear warp
# Usage
convertwarp

In [ ]:
%%bash

# Concatenate
convertwarp --ref=MNI152_T1_2mm_brain --premat=func2T1_6dof.mat --warp1=T1_2_MNI_warp --out=func2MNI_warp

In [ ]:
%%bash

# Apply concatenated transform
applywarp --ref=MNI152_T1_2mm_brain --in=example_func_brain --out=func2MNI --warp=func2MNI_warp

In [ ]:
%%bash

# Let's take a look in fslview
fslview MNI152_T1_2mm_brain.nii.gz func2MNI.nii.gz

In [ ]:
# Plot a more static image with edge overlay
mni="MNI152_T1_2mm.nii.gz"
fimg="func2MNI.nii.gz"
display=plotting.plot_anat(mni, display_mode="z", cut_coords=10, title="func to MNI with 6DOF func2T1 and FNIRT T1 to MNI")
display.add_edges(fimg)

## Invert transform of functional to MNI and apply to a mask

- So far we have gone in this direction: func -> T1 -> MNI  
- We can invert our transforms to go in the opposite direction: func <- T1 <- MNI
- The mask we are using comes from the [Harvard-Oxford sub-cortical atlas](http://neuro.imm.dtu.dk/wiki/Harvard-Oxford_Atlas#Subcortical)

In [ ]:
%%bash

# We need to invert our warp from T1_2_MNI to get the MNI_2_T1 warp
# Generic usage: invwarp -w A2B_warp -o B2A_warp -r A
invwarp -w T1_2_MNI_warp -o MNI_2_T1_warp -r my_T1_brain.nii.gz

In [ ]:
%%bash

# We can invert our func2T1_6dof.mat with convert_xfm
# Try this below, name the inverted matrix T12func.mat
convert_xfm -omat T12func_6dof.mat -inverse func2T1_6dof.mat

In [ ]:
%%bash

# Extracting an atlas-based region of interest in MNI space
# Copy the Harvard-Oxford sub-cortical atlas from fsl's library to our working directory
cp /usr/share/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz HarvardOxford-sub-maxprob-thr50-2mm.nii.gz

In [ ]:
%%bash
# change the image type to Mask/Label in fslview (remember where to look?)
# also change the colorscale from grayscale to render or random-rainbow
fslview HarvardOxford-sub-maxprob-thr50-2mm.nii.gz

In [ ]:
%%bash

# Example for extracting the left putamen (intensity 6)
# review: what does -bin do?
fslmaths HarvardOxford-sub-maxprob-thr50-2mm.nii.gz -thr 6 -uthr 6 -bin Lputamen.nii.gz

In [ ]:
%%bash

# Did it work?
fslview MNI152_T1_2mm_brain.nii.gz Lputamen.nii.gz

In [ ]:
%%bash

# Finally we can use apply warp to apply both our nonlinear and affine warps at once to our functional
# This is typically done when we want to put a regional mask in standard space into functional space 
# e.g., for our atlas-based ROI of the caudate
# interp=nn means the interpolation method we used is nearest neighbor.
# nearest neighbor: this means a voxel takes the value of it's closest neighbor
applywarp -i Lputamen -r example_func_brain -o Lputamen_mask_func -w MNI_2_T1_warp --postmat=T12func_6dof.mat --interp=nn

In [ ]:
%%bash

# Did it work? 
fslview example_func.nii.gz Lputamen_mask_func.nii.gz

In [ ]:
# Plot a more static image with some transparency using alpha setting
fimg="example_func.nii.gz"
roi="Lputamen_mask_func.nii.gz"
plotting.plot_roi(roi,fimg, display_mode="z", cut_coords=10, alpha=.5, title="Lputamen to func")

Why can't we use this to extract the timeseries from this region in the my_bold.nii.gz image yet?

## Problem sets

*1a)* Use the epi_reg tool to generate a transform from functional to T1 space, and combine these transforms to complete a registration of the functional to MNI using BBR for the functional step.
* Name the output of bbr func2T1_bbr
* Name the output of the final concatenated warp func2MNI_bbr 

In [ ]:
%%bash

# Examine usage below
epi_reg

In [ ]:
%%bash

# The epi_reg command will take a little time because there appears to be a bug when you supply your own _wmseg
# Therefore it will run FAST 


*1b)*  Comment on whether you think BBR did a better job and why or why not. Feel free to use figures to help visualize your comparison. 

*2)* Extract a different region of interest from the Harvard Oxford sub-cortical atlas. Describe the anatomical region you chose and complete steps to register your mask to the functional bold image. You can use whichever transform set from above worked best.

Show your steps below and create a figure with nilearn to visualize your ROI on the reference functional image.

In [ ]:
%%bash

# Your code 

In [ ]:
# Code for your ROI figure (using the nilearn plotting tool)